# iLOrest examples


Version 0.72

## Introduction

This Jupyter Notebook contains iLOrest [examples](#BackToList) as well as tips and tricks answering recurrent questions from HPE customers.

## <a id = "EnvironmentPreparation">Environment preparation</a>

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation (Version: 0.30) ############


# Set Student ID number
export stdid={{ STDID }}
Id=$(id --user --name)

# location and ports variables
ObmcBasePort={{ OBMCBASEPORT }}
iLO5DlBasePort=45000
iLO5SyBasePort=46000
RMCSdfBasePort=47000

let OpenBmcPort=${ObmcBasePort}+${stdid}
let iLO5DlSimulatorPort=$iLO5DlBasePort+${stdid}
let iLO5SySimulatorPort=${iLO5SyBasePort}+${stdid}
let RMCSdfSimulatorPort=${RMCSdfBasePort}+${stdid}
let ilo5Port=443

CacheDlDir="${PWD}/iLOrestCacheDl"
CacheSyDir="${PWD}/iLOrestCacheSy"
CacheSdfDir="${PWD}/iLOrestCacheSdf"
LogDir="${PWD}"
LogFile="${LogDir}/iLOrest.log"

iLO5DlSimulatorIP=ilo5simulators
iLO5SySimulatorIP=ilo5simulators
RMCSdfSimulatorIP=ilo5simulators
OpenBmcIP=openbmcsimulators

iLO5DlSimulator=${iLO5SimulatorIP}:${iLO5DlSimulatorPort}
iLO5DlSimulatorURI=https://${iLO5DlSimulator}

iLO5SySimulator=${iLO5SimulatorIP}:${iLO5SySimulatorPort}
iLO5SySimulatorURI=https://${iLO5SySimulator}

RMCSdfSimulator=${RMCSdfSimulatorIP}:${RMCSdfSimulatorPort}
RMCSdfSimulatorURI=https://${RMCSdfSimulator}

OpenBmc="${OpenBmcIP}:${OpenBmcPort}"
OpenBmcURI="https://${OpenBmc}"

ilo5IP="ilo5"
ilo5="${ilo5IP}:${ilo5Port}"
ilo5URI="https://${ilo5}"

# Credentials
User="student"
Password={{ REDFISHPWD }}

# Miscellaneous
w=$(basename $PWD)
alias ResetSimulators="../create-globalbmc.shc.x &>/dev/null ; sleep 1"

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in OpenBmc ilo5 iLO5DlSimulator iLO5SySimulator RMCSdfSimulator ; do
    ip="${bmc}IP" ; port="${bmc}Port"
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null &&
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

echo
# Retrieve iLO firmware versions from ServiceRoot (no credentials needed)
for bmc in ilo5 iLO5DlSimulator iLO5SySimulator ; do
    ip="${bmc}IP" ; port="${bmc}Port"
    echo -n "$bmc firmware version: "
    curl --silent --insecure -X GET https://$(eval echo \$${ip}):$(eval echo \$${port})/redfish/v1 | \
         jq  '[.Oem.Hpe.Manager[]] | .[] | .ManagerFirmwareVersion' 2>/dev/null
done

echo 
# Print ilorest version
ilorest --version

### Cache preparation

The following examples can be run against either an HPE ProLiant DL360 Gen10 simulator or an HPE Sy480 Gen10 simulators. To allow these simulateneous tests, we need to populate a cache directory for each simulator with the corresponding mockup and then create aliases of the `ilorest` command with the ad'hoc cache directory supplied on the command line.

Moockup files have been created with the [DMTF Mockup creator utility](https://github.com/DMTF/Redfish-Mockup-Creator).

In [ ]:
# iLOrest cache preparation
iLO5SyCache="iLO5Sy480g10.tgz"
iLO5DlCache="iLO5Dl360g10.tgz"
RMCSdfCache="RMCSdf.tgz"

CacheLocation="${PWD}" 
echo
for s in iLO5Dl iLO5Sy RMCSdf ; do
   case $s in 
      RMC*) ServerType=$(echo $s | cut -dC -f2); ;;
      iLO*) ServerType=$(echo $s | cut -d5 -f2); ;;
      *) echo "Server $s not known" ; break; ;;
   esac
   
   BasePort="${s}BasePort" ; SimulIP="${s}SimulatorIP"; SimulPort="${s}SimulatorPort"
   exp="s?$(eval echo \$$BasePort)?$(eval echo \$$SimulPort)?"
   
   CacheDir="${CacheLocation}/$(eval echo \$$SimulIP):$(eval echo \$$SimulPort)"
   Cache=$(eval echo "\$${s}Cache")
   
   [ -d ${CacheDir} ] && rm -rf ${CacheDir}                       &>/dev/null
   mkdir -p ${CacheDir}/cache/                                    &>/dev/null
   tar -C ${CacheDir}/cache -x -f ${Cache} 
   sed -i -e 's/:'$(eval echo \$${s}BasePort)'/:'$(eval echo \$${s}SimulatorPort)'/' ${CacheDir}/cache/*
   alias ilorest${ServerType}="ilorest --nologo --cache-dir ${CacheDir} --logdir ${LogDir}"
   echo -e "\tPopulated $s cache directory under $(basename ${CacheDir}) and created ilorest${ServerType} alias"   
done

### <a id = "iLO5SimulatorRestart">Restart iLO 5 simulator</a>

If you need or desire to restart your iLO 5 simulator to start this workshop from scratch or for other reasons, run the following cell at any time.

In [ ]:
# Simulators restart
ResetSimulators

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in ilo5 OpenBmc iLO5DlSimulator iLO5SySimulator RMCSdfSimulator ; do
    ip="${bmc}IP" ; port="${bmc}Port"
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &>/dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

## <a id = "BackToList">List of examples</a>

Note that the list of examples present in this Jupyter Notebook is changing over time. The classification is not following any strict model; it is a mix of Root service and data type locations.


<table>
    <tr>
        <th> &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; </th>
        <th> &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; </th>
    </tr>
    <tr>
        <td>    
            
Account Service
   - [iLO users: Enforce password complexity and minimum password length](#EnforceiLOUsersPasswordComplexity)
        
        </td>
        <td>
            
Chassis
   - [Chassis Serial Number](#ChassisSerialNumber)
   - [Power State](#PowerState)
   - [Indicator LED](#IndicatorLED)
   - [Chassis Asset Tag](#ChassisAssetTag)
            
        </td>
    </tr>
    <tr>
        <td>
            
Thermal
   - [Server thermal information](#ThermalInfo)
   - [Thermal configuration](#ThermalConfiguration)
            
        </td>
        <td>
            
Managers (iLO)
   - [Retrieve iLO firmware version](#iLOFwVersion)
   - [Enable/Disable SNMPv1 in iLO](#SNMPv1Enabled)

        </td>
    </tr>
    <tr>
        <td>
            
CompusterSystem
   - [PostState and DiscoveryComplete](#PostState_And_DiscoveryComplete)
   - [AMS and HostOS](#AMS_And_HostOS)
   - [Bios/RBSU Login password](#ManageBiosRbsuPassword)
   - [Computer System Asset Tag](#ComputerSystemAssetTag)
                       
        </td>
        <td>
            
Resource Directory
   - [Introduction](#ResourceDirectoryIntroduction)             
             
        </td>
    </tr>
    <tr>
        <td>
            
Security
   - [Security Dashboard](#SecurityDashboard)
   - [iLO Security Mode/State](#iLOSecurityMode)
   - [Login Security Banner](#LoginSecurityBanner)

        </td>
        <td>
            
Bios
   - [Server Serial Number](#BiosSerialNumber)
                      
        </td>       
    </tr>
    <tr>
        <td>
            
Storage
   - [SSD Wear Status](#SSD_WearStatus)            
            
        </td>
        <td>
            
Logout
   - [Logout](#Logout)            
            
        </td>
    </tr>
</table>

## Account Service

This root service models the iLO User Accounts.

### <a id = "EnforceiLOUsersPasswordComplexity">iLO users: Enforce password complexity and minimum password length</a>

[API Reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#oem-hpe-enforcepasswordcomplexity) keyword: `EnforcePasswordComplexity` \
iLO GUI location: `Security - Access Settings`

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select DMTF data type 
eval ilorest$s select AccountService

# List properties of the HPE Oem extension in json format 
eval ilorest$s list Oem/Hpe --json

# Modify two AccountService parameters
eval ilorest$s set Oem/Hpe/EnforcePasswordComplexity=True
eval ilorest$s set Oem/Hpe/MinPasswordLength=5

# Retrieve the list of modified paramters
echo -e "\n\nList of modified parameters:"
eval ilorest$s status

# Uncommnent the following comand to commit changes in the iLO 5 simulator.
# Note that you will not get the same answer as if done on a real iLO 5.
#eval ilorest$s commit

# Undo changes (clear cache)
eval ilorest$s select AccountService --refresh

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Chassis

This section contains parameters related to the `Chassis` [data type](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#chassis-v1_6_0-chassis). A Chassis represents a physical or virtual container of compute resources with attrbutes such as Field Replacement Units (FRU) information, power supplies, temperature, etc.

Complex chassis subsystems may have their own data type (i.e. [Thermal](#ThermalInfo))

Chassis examples:
  - Rack
  - Blade enclosure
  - HPE Synergy frame 
  - Compute node chassis

### <a id = "ChassisSerialNumber">Chassis Serial Number</a>

The chassis Serial Number (SN) of an HPE compute node mentioned on an external label/tag with code bar/QRcode is identical to its embedded server serial number (Bios parameter). Note that the serial number of a C7000 enclosure chassis or Synergy frame chassis is unique and different to every SN of embedded compute nodes.

The following cell retrieves the serial number of an HPE compute node chassis and shows explicitely that it is a `read-only` property. The server serial number is presented [here](#BiosSerialNumber) in the documentation.

This chassis serial number is also present in the Bios data type as a writable resource. See the [example](#BiosSerialNumber) below.

[API Reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#serialnumber294) keyword: `SerialNumber` \
iLO GUI location: `Information - iLO Overview` 



In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select Chassis

# Get SerialNumber from the Chassis data type
eval ilorest$s get SerialNumber

# Note that this property is READ-ONLY
echo -e "\n\nGet information about resource:"
eval ilorest$s info SerialNumber

### <a id = "PowerState">Power State</a>

The following cell retrieves the power state and information related to this parameter. Note that this parameter is read-only. If you want a more precise server state, read the [PostState and DiscoveryComplete](#PostState_And_DiscoveryComplete) and [AMS and HostOS](#AMS_And_HostOS) paragraphs.

As mentioned in the [API reference documentation](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#powerstate) this property is part of the `Chassis` data type. 

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select Chassis

# Get property value
eval ilorest$s get PowerState

# Retrieve property info with possible values.
echo -e "\n\nGet information about resource:"
eval ilorest$s info PowerState



### <a id = "IndicatorLED">Indicator LED</a>

The chassis indicator LED corresponds to the blue LED that is used to identify the chassis of a compute node or of a Synergy framce in the datacenter.

The following cell prints the status of the `IndicatorLED` property of the compute node only (`RackMount` or `Blade`). It filters out the `IndicatorLED` status of the Synergy frame if any.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
#s="Dl" 
s="Dl"

# Set filter depending on Chassis type
case $s in  Dl )  Filter="RackMount"; ;; Sy ) Filter="Blade"; ;; * )  echo "Chassis Type not recognized"; ;; esac

# Select data type
eval ilorest$s select Chassis

# Get property value
IndicatorLED=$(eval ilorest$s get --json IndicatorLED --filter ChassisType="$Filter")
echo "IndicatorLED=$IndicatorLED"

# Retrieve property info
eval ilorest$s info IndicatorLED

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Set filter depending on Chassis type
case $s in  Dl )  Filter="RackMount"; ;; Sy ) Filter="Blade"; ;; * )  echo "Chassis Type not recognized"; ;; esac

# Refresh cache
eval ilorest$s select Chassis --refresh

# Change the indicator LED state
LEDState=$(eval ilorest$s get IndicatorLED --filter ChassisType="$Filter" | cut -d= --field=2)
echo -e "LED State:$LEDState\n"  

if [ "$LEDState" = "Off" ]; then
    LEDState="Lit"
else
    LEDState="Off"
fi

# Change property and commit
echo "Changing IndicatorLED to $LEDState"
eval ilorest$s set IndicatorLED="$LEDState" --filter ChassisType="$Filter"  #--commit 
echo

# Validate property change
#eval ilorest$s get IndicatorLED

### <a id = "ChassisAssetTag">Chassis Asset Tag</a>

The Chassis `AssetTag` is different from the `ComputerSystem` [Asset Tag](#ComputerSystemAssetTag). 

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select Chassis --refresh

# Get property value
eval ilorest$s get AssetTag

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select Chassis --refresh

# Change the Chassis Asset Tag property. Uncomment the --commit when performing this action against a real BMC
eval ilorest$s set AssetTag="New\ Chassis\ Asset\ Tag" #--commit 

# Verify change
echo
eval ilorest$s get AssetTag

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Thermal

The [Thermal](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#thermal-v1_1_0-thermal) data type describes the system thermal metrics. It provides the temperatures in different locations in the chassis as well as fan properties. It also offers the possibility to configure the fans with a specific cooling directive (Optimal, Enhanced, etc ...). 

Thermal information belongs to the `Thermal` data type. However, the thermal configuration (`ThermalConfig`) is part of the `Bios` data type. Hence, to modify the `ThermalConfig` property, you will need to reboot the entire system to effectively apply this modification.

Starting with iLO firmware **2.30**, it is theoritically possible to modify the thermal configuration using the `Oem/Hpe/ThermalConfiguration` property of the `Thermal` data type without any system reset. 

>**NOTE**: As of the writing of this workshop, `Oem/Hpe/ThermalConfiguration` property cannot be modified (Read-Only property error). Hence you need to use a `rawpatch` command to modify it. Possible values for the `ThermalConfiguration` property are: `OptimalCooling`, `IncreasedCooling`, `MaximumCooling` and `EnhancedCooling`. The last two values differ from the Bios `ThermalConfig` property (`MaxCooling`, `EnhancedCPUCooling`)

### <a id = "ThermalInfo">Server Thermal Information</a>

The following cell retrieves both fan and sensor properties, including temperatures.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Retrieve server thermal properties
eval ilorest$s list --json --select Thermal

If you want only sensor information, you can use the **`serverinfo`** macro command like that found in the following cell

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Retrieve data type used by the server info macro command
eval ilorest$s serverinfo --thermal

### <a id = "ThermalConfiguration">ThermalConfiguration</a>

The following cells retrieve the thermal configuration from the `Bios` data type first, and then from the `Thermal` data type.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Sy" 

# Select data type
eval ilorest$s select Bios.

# Retrieve property
eval ilorest$s get Attributes/ThermalConfig
echo

# Retrieve possible values for the ThermalConfig property
#   NOTE: The following command fails against an iLO 5 simulator because the 
eval ilorest$s info Attributes/ThermalConfig 

# Change ThermalConfig
echo -e "\n\nChanging ThermalConfig"
eval ilorest$s set Attributes/ThermalConfig="IncreasedCooling"  #--commit

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Verify iLO firmware is equal or greater than 2.30
eval ilorest$s get FirmwareVersion --select Manager.
echo

# Select data type
eval ilorest$s select Thermal

# Retrieve property
eval ilorest$s get Oem/Hpe/ThermalConfiguration --json | jq -r '.[]|.[]'
echo

# Retrieve possible values for property
eval ilorest$s info Oem/Hpe/ThermalConfiguration
echo

# Modify property
eval ilorest$s select Thermal --refresh
eval ilorest$s set Oem/Hpe/ThermalConfiguration="MaximumCooling"



<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Managers

Examples in this section are related to the `Manager` data type as well as HPE Oem data types located below the `/redfish/v1/Managers/` end point like the `HpeiLOSnmpService`. 

### <a id = "iLOFwVersion">Retrieve iLO firmware version</a>

The easiest way to retrieve an iLO firmware version is from the `Oem.Hpe.ManagerFirmwareVersion` property of the ` ServiceRoot` data type at `/redfish/v1`. See the last loop in the enviroment preparation cell above. Note that a GET on this location does not require any authentication. 

As pert the [DMTF Schema documenation](http://redfish.dmtf.org/schemas/v1/Manager.v1_4_9.json#/definitions/Manager), the standard location for managers firmware is in the  [FirmwareVersion](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#firmwareversion1267) property of the [Manager](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#manager-v1_5_1-manager) data type.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select Manager.

# Get current value
eval ilorest$s get FirmwareVersion
echo

### <a id = "SNMPv1Enabled">Enable/Disable SNMPv1 in iLO</a>

> **NOTE** Changing the `SNMPv1Enabled` setting appeared in iLO 5 FW **2.30**.

    API Reference document keyword: "SNMPv1Enabled" 
    iLO GUI location: Management --> SNMP Settings --> SNMP Alerts --> SNMPv1

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Sy" 

# Select data type
eval ilorest$s select HpeiLOSnmpService

# Get current value
eval ilorest$s get SNMPv1Enabled
echo

# Set new value
eval ilorest$s set SNMPv1Enabled=False #--commit

#eval ilorest$s status

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## HPE iLO Resource Directory

The HPE iLO Resource Directory is an HPE OEM extention that contains the exhaustive list of the data types present in a Redfish service. This directory can be used by Redfish clients to lookup resources faster. Without this extension, Redfish clients must crawl the entire Redfish tree, starting at the root entry point, to find a specific resource.

The iLOrest utility caches the HPE iLO Resource Directory locally during the login process. When you select a data type, iLOrest very quickly retrieves its location from the cached directory. Then, when you submit get or set requests against specific properties, iLOrest starts looking for them starting at the location found in the cached directory.


### <a id = "ResourceDirectoryIntroduction">Introduction</a>

The `HpeiLOResourceDirectory` data type contains an array called `Instances`. Each element of this array corresponds to a data type with associated descriptors: `@odata.id` and `HttpMethods`. The first descriptor is the entry point of the data type in the Redfish tree and the second lists the valid HTTP methods against the data type.

> NOTE: The word "instance" is a synonym of "data type". You will find it in DMTF [python programs](https://github.com/DMTF/Redfish-Tacklebox/blob/master/redfish_utilities/sensors.py) as well as [HPE programs](https://github.com/HewlettPackard/python-ilorest-library/blob/master/examples/Redfish/get_resource_directory.py).

The following cell retrieves the first data types (aka instances) of the current iLO 5.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select HpeiLOResourceDirectory

# List the first elements of array Instances containing data type properties
eval ilorest$s list Instances --json | head -30

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Computer System

This section contains examples related to the `ComputerSystem` data type.

### <a id = "PostState_And_DiscoveryComplete">PostState and DiscoveryComplete</a>

Use these state parameters to evaluate the current state of a system before modifying or viewing its resources. As explained in this [article](https://developer.hpe.com/blog/master-the-redfish-server-states-to-improve-your-monitoring-and-manageme), certain parameters cannot be modified if the computer is not in a specific state.

Moreover, some parameters cannot be viewed when the entire discovery of all subsystems is not complete. As an example, you will not be able to view or modify SmartArray and logical drive configurations if the `SmartArrayDiscovery` is not complete.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select ComputerSystem

# Retrieve and print DeviceDiscoveryComplete and PostState properties
eval ilorest$s get Oem/Hpe/DeviceDiscoveryComplete | grep -v -E 'Oem|Hpe'; echo
eval ilorest$s get Oem/Hpe/PostState | grep -v -E 'Oem|Hpe' ; echo

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

### <a id = "AMS_And_HostOS">AMS and HostOS</a>

When the HPE Agentless Management Service (AMS) is installed in the operating system of the host, it is possible to get various information from the `HostOS` property. This property is present only when AMS is up and running in the host OS.

Use keywords `agentless` or `ams` in the [API reference documentation](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#introduction) to get the data type related to this topic.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Print info on the AgentlessManagementService
echo "*********************************************************************"
eval ilorest$s info Oem/Hpe/AggregateHealthStatus/AgentlessManagementService
echo "*********************************************************************"
echo -e "\n\n"


# Select type
eval ilorest$s select ComputerSystem.

# Retrieve and print AMS status
AmsStatus=$(eval ilorest$s get --json Oem/Hpe/AggregateHealthStatus/AgentlessManagementService | \
jq -r '.Oem.Hpe.AggregateHealthStatus.AgentlessManagementService')
echo "AMS Status on $s simulator: $AmsStatus" ; echo

# Retrieve and print HostOS info only if AMS is ready
if [ "$AmsStatus" == "Ready" ] ; then
    echo "Host OS properties:"
    eval ilorest$s get --json Oem/Hpe/HostOS | jq -r '.Oem.Hpe.HostOS'
fi

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

### <a id = "ManageBiosRbsuPassword">Bios/RBSU Login password</a>

The following cell uses the `setpassword` macro command to set the RBSU password for the first time and then removes it.

> **NOTE**: The `setpassword` command may return errors when launched against a DMTF iLO 5 simulator.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Retrieve help syntax and examples
echo -e "setpassword help syntax:\n"
eval ilorest$s help setpassword


# Setting a password for the first time
echo -e "\nSetting a password for the first time"
eval ilorest$s setpassword --newpassword testnew --currentpassword None

# Removing the Bios/RBSU password
echo -e "\n\nRemoving the BIOS/RBSU password"
eval ilorest$s setpassword --newpassword None --currentpassword testnew 

# No commit needed. No iLO nor system reset needed. 
eval ilorest$s status


<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

### <a id = "ComputerSystemAssetTag">ComputerSystem Asset Tag</a>

The `ComputerSystem` Asset Tag is different from the `Chassis` [Asset Tag](#ChassisAssetTag).

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type and clear cache
eval ilorest$s select ComputerSystem. --refresh

# Retrieve ComputerSystem Asset Tag
eval ilorest$s get AssetTag

# Change ComputerSystem Asset Tag
eval ilorest$s set AssetTag="New\ ComputerSystem\ Asset\ Tag" --commit 
echo

# Verify change
#eval ilorest$s get AssetTag

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Security

### <a id = "SecurityDashboard">Security Dashboard</a>

The following five examples retrieve different security dashboard properties.

#### Overall security dashboard status

Use keyword `Security Dashboard` or `SecurityDashboard` in the [API reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#hpeilosecuritydashboard-v1_0_0-hpeilosecuritydashboard) to find the overall security dashboard data type.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select HpeiLOSecurityDashboard.

# Retrieve overall security dashboard status 
eval ilorest$s get OverallSecurityStatus 

#### All Security parameters and properties in json format

Use keyword `SecurityParam` to find the corresponding data type in the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#hpeilosecurityparam-v1_1_0-hpeilosecurityparam).

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select HpeiLOSecurityParam.

# List all security params in JSON format. Discard "@odata" metadata to get cleaner output.
eval ilorest$s list --json | grep -v '@'

#### List only security params at risk

To retrieve this list you don't need to select the data type as it has already selected in the previous cell.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select HpeiLOSecurityParam.

eval ilorest$s list --filter SecurityStatus="Risk" | grep -v '@'
echo

#### Get specific security parameter properties

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select HpeiLOSecurityParam.

eval ilorest$s get --filter Name="Require\ Login\ for\ iLO\ RBSU" | grep -v '@'
echo

#### Get specific property of a specific security parameter

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select HpeiLOSecurityParam.

# Retrieve the `Ignore` property of the `Minimum Password Length` security parameter
eval ilorest$s get Ignore --filter Name="Minimum\ Password\ Length"
echo

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

### <a id = "iLOSecurityMode">Security Mode/State</a>

From the iLO 5 Graphical User Interface, in the `Security` --> `Encryption` menu, you can view and modify the `SecurityState` setting. This setting is part of the `HpeSecurityService` data type as mentioned in the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#securitystate).

>NOTE: Software like the Smart Update Tool (`SUT`) may refer to this setting as `Security Mode` instead of `Security State`. 

#### Retrieve and modify the iLO 5 Security State setting

The following cell retrieves all the properties of the `HpeSecurityService` and modifies the `SecurityState`. A similar change on a real iLO 5 **automatically triggers a reset** of the iLO. 

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type and refresh cache
eval ilorest$s select HpeSecurityService. --refresh

# Retrieve all Security Service paramters
eval ilorest$s get --json

# Modify Security State
SecurityState=$(eval ilorest$s get SecurityState | cut --delimiter== --field=2)
echo 

if [ "$SecurityState" = "Production" ]; then
   echo "Setting Security State to HighSecurity"
   eval ilorest$s set SecurityState="HighSecurity" #--commit
else
    echo "Setting Security State to Production"
    eval ilorest$s set SecurityState="Production" #--commit
fi

echo
eval ilorest$s get SecurityState 

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

### <a id = "LoginSecurityBanner">Login Security Banner</a>

From the iLO 5 Graphical User Interface, in the `Security` --> `Login Security Banner` menu, you can enable/disable and customize the `LoginSecurityBanner` setting. This setting is part of the `HpeSecurityService` data type as mentioned in the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#loginsecuritybanner).

#### Retrieve and modify the iLO 5 Security State setting

The following cell modifies the state of the Security Login Banner and provides a message when enabled. You don't need to reset the iLO 5 to effectively modify this banner.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Sy" 

# Select data type and refresh cache
eval ilorest$s select HpeSecurityService. --refresh

# Retrieve Login Security Banner settings
eval ilorest$s get LoginSecurityBanner --json

# Modify Login Security Banner settings
LoginSecurityBanner=$(eval ilorest$s get LoginSecurityBanner/IsEnabled | cut --delimiter== --field=2)
echo 

if [ $LoginSecurityBanner = False ]; then
   echo "Enabling Login Security Banner with a custom message"
   eval ilorest$s set LoginSecurityBanner/IsEnabled=True #--commit
   eval ilorest$s set LoginSecurityBanner/SecurityMessage="This is My Custom Security Message" #--commit
else
    echo "Disabling Login Security Banner"
    # The following set command is not required with a real iLO 5. 
    # It's only there to get a "realistic" behavior of the simulator.
    eval ilorest$s set LoginSecurityBanner/SecurityMessage="" #--commit
    eval ilorest$s set LoginSecurityBanner/IsEnabled=False    #--commit
fi

echo
eval ilorest$s get --json LoginSecurityBanner 

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Bios

### <a id = "BiosSerialNumber">Server Serial Number stored in BIOS</a>

In a computer node, the Serial Number stored in the `Bios.` data type corresponds to the server mother board serial number. HPE requires the change of this serial number in case of mother board replacement to match the chassis serial number. 

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Dl" 

# Select data type
eval ilorest$s select Bios. --refresh
echo

# Make sure SN is read-write
eval ilorest$s info Attributes/SerialNumber --latestschema
echo

# Get SerialNumber from Bios data type.
eval ilorest$s get Attributes/SerialNumber

echo "Set new SN"
eval ilorest$s set Attributes/SerialNumber="NEWSN" #--commit

# Reload a fresh simulator cache
eval ilorest$s select Bios. --refresh
echo 

# Validate the SN change
eval ilorest$s get Attributes/SerialNumber
echo

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Storage

The following cell lists the storage related data types implemented in the current iLO 5 Redfish service.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Sy" 

eval ilorest$s types | grep -i -E 'storage|drive' | grep -v -i driver

### <a id = "SSD_WearStatus">SSD Wear Status</a>

The `WearStatus` property is an HPE Oem property part of the `Drive` data type as mentioned in the [API Reference Document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#oem-hpe-wearstatus). Note that the Redfish standard proposes the `PredictedMediaLifeLeftPercent` property, also in the `Drive` type, and being somehow equivalent.

The `Drive` data type may be absent in servers not containing suitable hardware. The physical DL360 Gen10 server available in this infrastructure (and the associated simulator) does not contain any SSD drive with a `WearStatus`. Hence, you will have to use the Synergy iLO 5 simulator to view this property.

 &nbsp;


The Redfish service of the Synergy iLO 5 simulator contains two SSD drives. The next cell retrieves successively all their properties, the `PredictedMediaLifeLeftPercent` standard property and then the Oem HPE `WearStatus` property.

In [ ]:
# Select type of server: Dl (ProLiant DL360g10) or Sy (Synergy)
s="Sy" 

echo -e "Drive list. Physical and Logical Hpe SmartStorage excluded"
eval ilorest$s list --json --select Drive. --filter "@odata.type"="#Drive.v1_1_0.Drive"

echo -e "\n\n'PredictedMediaLifeLeftPercent' standard property of drives. Odata metadata discarded."
eval ilorest$s get --json Id PredictedMediaLifeLeftPercent --filter "@odata.type"="#Drive.v1_1_0.Drive"

echo -e "\n\nOem/Hpe specific 'WearStatus' of non-SmartStorage" 
eval ilorest$s get Id Oem/Hpe/WearStatus --json --filter "@odata.type"="#Drive.v1_1_0.Drive"

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## <a id = "Logout">Logout</a>

Always logout when your session is over. You can as well logout and [restart](#iLO5SimulatorRestart) the iLO 5 simulator if you want once again use this Jupyter Notebook.

In [ ]:
ilorestDl logout
ilorestSy logout

<-- [Environment Preparation](#EnvironmentPreparation) ; [List of examples](#BackToList)  ; [Restart iLO 5 Simulator](#iLO5SimulatorRestart) ; [Logout](#Logout) -->

## Summary

In this workshop, you discovered several HPE eval iLOrest$s examples. Go to the [Conclusion Notebook](3-Conclusion.ipynb) if you are finished.